<a href="https://colab.research.google.com/github/jjangmo91/ParkLab/blob/main/project/2025%20NASA%20Space%20Apps%20Challenge/04_ASF_SAR_DAAC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install asf_search -q

In [ ]:
# NASA Earthdata 로그인 정보 설정
import getpass
import os

# .netrc 파일 경로 설정
netrc_path = os.path.expanduser("~/.netrc")

# 사용자에게 아이디와 비밀번호를 안전하게 입력받음
username = input("Enter your NASA Earthdata Username: ")
password = getpass.getpass("Enter your NASA Earthdata Password: ")

# .netrc 파일 작성
# Colab 환경의 홈 디렉토리에 인증 정보를 저장합니다.
with open(netrc_path, "w") as f:
    f.write(f"machine urs.earthdata.nasa.gov login {username} password {password}\n")

# 파일 권한 설정 (보안 강화)
os.chmod(netrc_path, 0o600)

print("\nNASA Earthdata 인증 정보 설정 완료!")

In [ ]:
import ee
import geemap

# Earth Engine 인증
ee.Authenticate()
# Earth Engine 초기화
ee.Initialize(project='ee-jjangmo91') # 본인 계정

In [ ]:
import subprocess
import zipfile
import glob
import pandas as pd
import asf_search as asf
from shapely.geometry import box

# ==============================================================================
# 1단계: 분석 파라미터 설정 (이 부분만 수정해서 사용하세요)
# ==============================================================================

# 분석 지역 (Area of Interest)
# min_lon, min_lat, max_lon, max_lat
aoi_bounds = (103.0, 9.5, 107.0, 13.5)

# 분석 기간
start_date = '2006-01-01'
end_date = '2008-12-31'

# 검색할 위성 및 데이터 종류
# (플랫폼: ALOS, RADARSAT-1, ERS-2, SENTINEL-1A 등)
platform = asf.PLATFORM.ALOS

# Colab 내에 다운로드한 파일을 저장할 폴더
download_folder = '/content/sar_downloads'
os.makedirs(download_folder, exist_ok=True)

# 업로드할 GEE Asset 폴더 (미리 GEE에서 만들어두세요)
# 예: users/USERNAME/mekong_sar
gee_asset_folder = 'projects/ee-jjangmo91/assets/mekong_sar_assets' # 본인 GEE Asset 폴더 경로로 변경

# ==============================================================================
# 2단계: ASF API로 데이터 검색
# ==============================================================================
print("="*50)
print(f"1. ASF에서 데이터 검색을 시작합니다...")
print(f"   - 기간: {start_date} ~ {end_date}")
print(f"   - 위성: {platform}")

aoi_wkt = box(*aoi_bounds).wkt
search_opts = {
    'platform': platform,
    'start': start_date,
    'end': end_date,
    'intersectsWith': aoi_wkt
}

results = asf.search(**search_opts)
print(f"--> 총 {len(results)}개의 영상을 찾았습니다.")
print("="*50)

# ==============================================================================
# 3단계: 검색된 데이터 다운로드
# ==============================================================================
if results:
    print(f"2. 데이터 다운로드를 시작합니다 (저장 폴더: {download_folder})")
    print("--> .netrc 파일에서 NASA Earthdata 인증 정보를 불러오는 중...")
    try:
        # .netrc 파일을 읽어 'urs.earthdata.nasa.gov'에 해당하는 인증 정보를 찾습니다.
        auth = netrc.netrc()
        username = auth.authenticators('urs.earthdata.nasa.gov')[0]
        password = auth.authenticators('urs.earthdata.nasa.gov')[2]
        print("--> 인증 정보 불러오기 성공!")
    except Exception as e:
        print(f" [오류] .netrc 파일에서 인증 정보를 찾을 수 없습니다: {e}")
        print("    -> 수동으로 사용자 이름과 비밀번호를 입력해주세요.")
        # .netrc 파일이 없을 경우 대비
        username = input("EDL 사용자 이름: ")
        password = getpass.getpass("EDL 비밀번호: ")

    session = asf.ASFSession()
    # asf_search 세션에 인증 정보를 전달합니다.
    session.auth.login(username, password)

    results.download(path=download_folder, session=session)
    print("--> 모든 파일 다운로드 완료!")
else:
    print("2. 다운로드할 데이터가 없습니다. 이 단계를 건너뜁니다.")

# ==============================================================================
# 4단계: GEE Asset으로 자동 업로드
# ==============================================================================
print(f"3. GEE Asset 업로드를 시작합니다 (대상 폴더: {gee_asset_folder})")

zip_files = glob.glob(os.path.join(download_folder, '*.zip'))

if not zip_files:
    print("--> 업로드할 파일이 없습니다.")
else:
    for zip_path in zip_files:
        print(f"\n처리 중인 파일: {os.path.basename(zip_path)}")

        target_tif = None
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            file_list = zip_ref.namelist()
            for fname in file_list:
                if ('_HH.tif' in fname or '_HV.tif' in fname or '_VV.tif' in fname) and '.aux.xml' not in fname:
                    zip_ref.extract(fname, download_folder)
                    target_tif = os.path.join(download_folder, fname)
                    print(f" --> 분석용 TIF 파일 찾음: {fname}")
                    break

        if target_tif:
            asset_name = os.path.splitext(os.path.basename(target_tif))[0]
            asset_id = f'{gee_asset_folder}/{asset_name}'

            print(f" --> GEE Asset으로 업로드 시작: {asset_id}")

            command = [
                'earthengine', 'upload', 'image',
                '--asset_id', asset_id,
                '--nodata_value', '0',
                target_tif
            ]

            try:
                subprocess.run(command, check=True)
                print(f" --> 업로드 작업(Task) 제출 완료! GEE UI의 Tasks 탭에서 진행 상황을 확인하세요.")
            except Exception as e:
                print(f" [에러] 업로드 실패: {e}")
        else:
            print(" --> 분석용 TIF 파일을 찾지 못했습니다.")

print("\n" + "="*50)
print("모든 파이프라인 작업이 완료되었습니다.")
print("="*50)